In [1]:
import os
import json
import pandas as pd
from pandas import json_normalize

In [2]:
json_dir = 'data/'  # Update this to your folder path
output_csv = 'ipl_ball_by_ball.csv'

In [3]:
all_dataframes = []

In [4]:
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        match_id = filename.replace('.json', '')  # Extract match ID from filename

        # Load JSON file
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Extract match-level info
        match_info = data['info']
        teams = match_info['teams']
        date = match_info['dates'][0]  # Take first date if multiple
        venue = match_info.get('venue', 'Unknown')
        toss_winner = match_info['toss']['winner']
        toss_decision = match_info['toss']['decision']

        # Process each inning
        for inning_idx, inning in enumerate(data['innings'], 1):
            team = inning['team']
            batting_team = team
            bowling_team = teams[1] if teams[0] == team else teams[0]  # Opponent team

            # Flatten deliveries for this inning
            deliveries = []
            for over in inning['overs']:
                over_number = over['over']
                for delivery in over['deliveries']:
                    delivery_data = {
                        'match_id': match_id,
                        'inning': inning_idx,
                        'batting_team': batting_team,
                        'bowling_team': bowling_team,
                        'over': over_number,
                        'ball': list(over['deliveries']).index(delivery) + 1,  # Ball number in over
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'non_striker': delivery['non_striker'],
                        'runs_batter': delivery['runs']['batter'],
                        'runs_extras': delivery['runs']['extras'],
                        'runs_total': delivery['runs']['total'],
                        'date': date,
                        'venue': venue,
                        'toss_winner': toss_winner,
                        'toss_decision': toss_decision
                    }

                    # Handle extras (wides, legbyes, etc.)
                    if 'extras' in delivery:
                        for extra_type, value in delivery['extras'].items():
                            delivery_data[f'extras_{extra_type}'] = value
                    else:
                        delivery_data['extras_wides'] = 0
                        delivery_data['extras_legbyes'] = 0
                        delivery_data['extras_byes'] = 0
                        delivery_data['extras_noballs'] = 0

                    # Handle wickets
                    if 'wickets' in delivery:
                        wicket = delivery['wickets'][0]  # Assume one wicket per ball
                        delivery_data['wicket_kind'] = wicket['kind']
                        delivery_data['player_out'] = wicket['player_out']
                        delivery_data['fielder'] = wicket['fielders'][0]['name'] if wicket.get('fielders') else None
                    else:
                        delivery_data['wicket_kind'] = None
                        delivery_data['player_out'] = None
                        delivery_data['fielder'] = None

                    deliveries.append(delivery_data)

            # Convert deliveries to DataFrame
            df = pd.DataFrame(deliveries)
            all_dataframes.append(df)

In [5]:
# Concatenate all dataframes
final_df = pd.concat(all_dataframes, ignore_index=True)

# Save to CSV
final_df.to_csv(output_csv, index=False)
print(f"Converted {len(all_dataframes)} matches to {output_csv}")

Converted 2227 matches to ipl_ball_by_ball.csv


In [9]:
final_df.head(8)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,runs_batter,...,toss_winner,toss_decision,extras_wides,extras_legbyes,extras_byes,extras_noballs,wicket_kind,player_out,fielder,extras_penalty
0,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1,DA Warner,TS Mills,S Dhawan,0,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN
1,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1,DA Warner,TS Mills,S Dhawan,0,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN
2,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,3,DA Warner,TS Mills,S Dhawan,4,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN
3,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1,DA Warner,TS Mills,S Dhawan,0,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN
4,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,5,DA Warner,TS Mills,S Dhawan,0,...,Royal Challengers Bangalore,field,2.0,NaN,NaN,NaN,None,None,None,NaN
5,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,6,S Dhawan,TS Mills,DA Warner,0,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN
6,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,7,S Dhawan,TS Mills,DA Warner,0,...,Royal Challengers Bangalore,field,NaN,1.0,NaN,NaN,None,None,None,NaN
7,1082591,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,S Dhawan,A Choudhary,DA Warner,1,...,Royal Challengers Bangalore,field,0.0,0.0,0.0,0.0,None,None,None,NaN


In [10]:
final_df.shape

(262148, 24)

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262148 entries, 0 to 262147
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   match_id        262148 non-null  object 
 1   inning          262148 non-null  int64  
 2   batting_team    262148 non-null  object 
 3   bowling_team    262148 non-null  object 
 4   over            262148 non-null  int64  
 5   ball            262148 non-null  int64  
 6   batter          262148 non-null  object 
 7   bowler          262148 non-null  object 
 8   non_striker     262148 non-null  object 
 9   runs_batter     262148 non-null  int64  
 10  runs_extras     262148 non-null  int64  
 11  runs_total      262148 non-null  int64  
 12  date            262148 non-null  object 
 13  venue           262148 non-null  object 
 14  toss_winner     262148 non-null  object 
 15  toss_decision   262148 non-null  object 
 16  extras_wides    256378 non-null  float64
 17  extras_leg

In [12]:
final_df.isnull().sum()

match_id               0
inning                 0
batting_team           0
bowling_team           0
over                   0
ball                   0
batter                 0
bowler                 0
non_striker            0
runs_batter            0
runs_extras            0
runs_total             0
date                   0
venue                  0
toss_winner            0
toss_decision          0
extras_wides        5770
extras_legbyes     10184
extras_byes        13526
extras_noballs     13102
wicket_kind       249133
player_out        249133
fielder           252743
extras_penalty    262146
dtype: int64

In [13]:
final_df.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'runs_batter', 'runs_extras',
       'runs_total', 'date', 'venue', 'toss_winner', 'toss_decision',
       'extras_wides', 'extras_legbyes', 'extras_byes', 'extras_noballs',
       'wicket_kind', 'player_out', 'fielder', 'extras_penalty'],
      dtype='object')